In [1]:
import pandas as pd
import re
imdb_df = pd.read_csv("data/imdb_data.csv")
imdb_df['_primaryTitle'] = imdb_df['primaryTitle'].str.lower()
imdb_df['_originalTitle'] = imdb_df['originalTitle'].str.lower()
imdb_df.rename(columns={'averageRating': 'imdb_rating', 'numVotes': 'imdb_numVotes'}, inplace=True)

metacritic_df = pd.read_csv("data/metacritic_data.csv")
metacritic_df['_title'] = metacritic_df['title'].str.lower()
metacritic_df.rename(columns={'user_rating': 'metacritic_rating', 'rating_count': 'metacritic_numVotes'}, inplace=True)
tmdb_df = pd.read_csv("data/tmdb_data.csv")
tmdb_df['_title'] = tmdb_df['title'].str.lower()
tmdb_df.rename(columns={'rating': 'tmdb_rating'}, inplace=True)


In [2]:
# clean metacritic data for parenthesis
def regex_filter(text):
    pattern = r'\((?:[a-z]{2}|\d{4})\)|marvel\'s\s*'  # Regex pattern to match two uppercase letters or four digits inside parentheses
    matches = re.findall(pattern, text)
    for match in matches:
        text = text.replace(match, '')  # Replace the matched pattern with an empty string
    return text.strip()

metacritic_df['_title'] = metacritic_df['_title'].map(regex_filter)
display(metacritic_df)
tmdb_df['_title'] = tmdb_df['_title'].map(regex_filter)

,id,title,metacritic_rating,metacritic_numVotes,release,_title
0,/tv/xavier-renegade-angel/,Xavier: Renegade Angel,9.3,16.0,2007,xavier: renegade angel
1,/tv/gintama/,Gintama,9,23.0,2006,gintama
2,/tv/foyles-war/,Foyle's War,9.8,15.0,2002,foyle's war
3,/tv/longmire/,Longmire,9.1,462.0,2012,longmire
4,/tv/better-call-saul/,Better Call Saul,9,3762.0,2015,better call saul
...,...,...,...,...,...,...
2538,/tv/one-tree-hill/,One Tree Hill,7.6,151.0,2003,one tree hill
2539,/tv/american-gods/,American Gods,7.1,476.0,2017,american gods
2540,/tv/veronica-mars/,Veronica Mars,7.3,281.0,2004,veronica mars
2541,/tv/them-2021/,Them,5.9,51.0,2021,them


In [3]:

merged_df = pd.merge(imdb_df, metacritic_df, left_on=['_primaryTitle', 'startYear'], right_on=['_title', 'release'], how='left', suffixes=('_imdb', '_metacritic'))

unmatched_metacritic_df = merged_df[merged_df['id'].isnull()].drop(columns=[col for col in merged_df.columns if '_imdb' in col])
print(unmatched_metacritic_df.shape)

if not unmatched_metacritic_df.empty:
    additional_matches = pd.merge(imdb_df, metacritic_df, left_on=['_originalTitle', 'startYear'], right_on=['_title', 'release'], how='inner', suffixes=('_imdb', '_metacritic'))
    unmatched_rows = merged_df[merged_df['tconst'].isnull()].drop(columns=[col for col in merged_df.columns if '_metacritic' in col])
    unmatched_rows = pd.concat([unmatched_metacritic_df, unmatched_rows])
    
print(unmatched_metacritic_df[unmatched_metacritic_df['tconst'] == 'tt9288030'])

print(unmatched_rows.shape)
#display(unmatched_rows.sort_values('numVotes',ascending=False))
display(unmatched_metacritic_df.sort_values('metacritic_numVotes',ascending=False))

metacritic_merged_df = merged_df.dropna()

print(metacritic_merged_df.shape)


(1345, 19)
Empty DataFrame
Columns: [tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres, imdb_rating, imdb_numVotes, _primaryTitle, _originalTitle, id, title, metacritic_rating, metacritic_numVotes, release, _title]
Index: []
(1345, 19)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,imdb_rating,imdb_numVotes,_primaryTitle,_originalTitle,id,title,metacritic_rating,metacritic_numVotes,release,_title
0,tt0142032,tvMiniSeries,Dune,Dune,0,2000,2000,265,"Adventure,Drama,Sci-Fi",6.9,23747,dune,dune,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0160904,tvSeries,MI-5,Spooks,0,2002,2011,60,"Action,Crime,Drama",8.3,17719,mi-5,spooks,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0181199,tvMiniSeries,Arabian Nights,Arabian Nights,0,2000,2000,175,"Adventure,Family,Fantasy",7.4,5491,arabian nights,arabian nights,NaN,NaN,NaN,NaN,NaN,NaN
5,tt0205700,tvSeries,Titus,Titus,0,2000,2002,23,"Comedy,Drama",7.6,6094,titus,titus,NaN,NaN,NaN,NaN,NaN,NaN
6,tt0206511,tvSeries,Even Stevens,Even Stevens,0,2000,2003,30,"Comedy,Family",7.6,15166,even stevens,even stevens,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3057,tt9817298,tvSeries,Selling Sunset,Selling Sunset,0,2019,\N,30,Reality-TV,6.5,7793,selling sunset,selling sunset,NaN,NaN,NaN,NaN,NaN,NaN
3061,tt9849210,tvSeries,Biohackers,Biohackers,0,2020,2021,44,"Drama,Sci-Fi,Thriller",6.8,11087,biohackers,biohackers,NaN,NaN,NaN,NaN,NaN,NaN
3062,tt9875006,tvMiniSeries,Flesh,Flesh,0,2020,2020,340,"Crime,Thriller",5.4,11548,flesh,flesh,NaN,NaN,NaN,NaN,NaN,NaN
3063,tt9879074,tvSeries,The Disappearance of Madeleine McCann,The Disappearance of Madeleine McCann,0,2019,2019,60,"Crime,Documentary,Mystery",6.6,11497,the disappearance of madeleine mccann,the disappearance of madeleine mccann,NaN,NaN,NaN,NaN,NaN,NaN


(1665, 19)


In [4]:
merged_df = pd.merge(imdb_df, tmdb_df, left_on=['_primaryTitle', 'startYear'], right_on=['_title', 'release'], how='left', suffixes=('_imdb', '_tmdb_df'))
unmatched_tmdb_df = merged_df[merged_df['id'].isnull()].drop(columns=[col for col in merged_df.columns if '_imdb' in col])
print(unmatched_tmdb_df.shape)
if not unmatched_tmdb_df.empty:
    additional_matches = pd.merge(imdb_df, tmdb_df, left_on=['_originalTitle', 'startYear'], right_on=['_title', 'release'], how='inner', suffixes=('_imdb', '_tmdb_df'))
    unmatched_rows = merged_df[merged_df['tconst'].isnull()].drop(columns=[col for col in merged_df.columns if '_tmdb_df' in col])
    unmatched_rows = pd.concat([unmatched_tmdb_df, unmatched_rows])
    

print(unmatched_rows.shape)
#display(unmatched_rows.sort_values('numVotes',ascending=False))
tmdb_merged_df = merged_df.dropna()

print(tmdb_merged_df.shape)
display(tmdb_merged_df)

(2011, 18)
(2011, 18)
(919, 18)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,imdb_rating,imdb_numVotes,_primaryTitle,_originalTitle,id,title,tmdb_rating,release,_title
3,tt0185906,tvMiniSeries,Band of Brothers,Band of Brothers,0,2001,2001,594,"Drama,History,War",9.4,506409,band of brothers,band of brothers,4613.0,Band of Brothers,85.00,2001.0,band of brothers
4,tt0204993,tvSeries,Dark Angel,Dark Angel,0,2000,2002,60,"Action,Drama,Mystery",7.4,34172,dark angel,dark angel,2407.0,Dark Angel,74.94,2000.0,dark angel
7,tt0207275,tvMiniSeries,The 10th Kingdom,The 10th Kingdom,0,2000,2000,417,"Adventure,Comedy,Family",8.3,19856,the 10th kingdom,the 10th kingdom,40546.0,The 10th Kingdom,76.19,2000.0,the 10th kingdom
10,tt0212671,tvSeries,Malcolm in the Middle,Malcolm in the Middle,0,2000,2006,22,"Comedy,Family",8.2,139516,malcolm in the middle,malcolm in the middle,2004.0,Malcolm in the Middle,84.87,2000.0,malcolm in the middle
11,tt0213327,tvSeries,Andromeda,Andromeda,0,2000,2005,43,"Action,Adventure,Drama",6.6,14594,andromeda,andromeda,2985.0,Andromeda,71.57,2000.0,andromeda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913,tt9794044,tvSeries,Citadel,Citadel,0,2023,\N,41,"Action,Drama,Thriller",6.1,31206,citadel,citadel,114922.0,Citadel,70.63,2023.0,citadel
2918,tt9813792,tvSeries,From,From,0,2022,\N,52,"Drama,Horror,Mystery",7.7,58832,from,from,124364.0,From,81.77,2022.0,from
2923,tt9827854,tvMiniSeries,Hollywood,Hollywood,0,2020,2020,347,"Drama,History",7.5,39824,hollywood,hollywood,87050.0,Hollywood,74.00,2020.0,hollywood
2924,tt9848536,tvSeries,We Are Who We Are,We Are Who We Are,0,2020,2020,462,Drama,7.3,9502,we are who we are,we are who we are,88713.0,We Are Who We Are,80.00,2020.0,we are who we are


In [5]:
trakt_df = pd.read_csv("data/trakt_data.csv")
trakt_df.rename(columns={'Rating': 'trakt_rating', 'Votes': 'trakt_numVotes'}, inplace=True)

merged_df = pd.merge(metacritic_merged_df, tmdb_merged_df, on='tconst')

merged_df = pd.merge(merged_df, trakt_df, on='tconst')
print(merged_df[merged_df['tconst'] == 'tt15426714'])

merged_df.rename(columns={'title_y': 'title', 'startYear_x': 'startYear',
                               'imdb_rating_x': 'imdb_rating', 'imdb_numVotes_x': 'imdb_numVotes',
                               }, inplace=True)
merged_df_final = merged_df[['tconst','title','startYear', 'imdb_rating', 
                             'imdb_numVotes','tmdb_rating','trakt_rating',
                             'trakt_numVotes', 'metacritic_rating', 'metacritic_numVotes']]

merged_df_final = merged_df_final.loc[merged_df_final.groupby(['title', 'startYear'])['trakt_numVotes'].idxmax()]
print(merged_df_final.columns)
print(merged_df_final[merged_df_final['tconst'] == 'tt9288030'])
merged_df_final = merged_df_final.drop_duplicates()

merged_df_final.to_csv('data/all_ratings_data.csv', index=False)


         tconst titleType_x primaryTitle_x originalTitle_x  isAdult_x  \
310  tt15426714    tvSeries     Wilderness      Wilderness          0   

     startYear_x endYear_x runtimeMinutes_x        genres_x  imdb_rating_x  \
310         2023      2023               \N  Drama,Thriller            6.3   

     ...  imdb_numVotes_y _primaryTitle_y _originalTitle_y      id_y  \
310  ...             8166      wilderness       wilderness  204276.0   

        title_y tmdb_rating  release_y    _title_y trakt_rating trakt_numVotes  
310  Wilderness        63.0     2023.0  wilderness           64            234  

[1 rows x 38 columns]
Index(['tconst', 'title', 'startYear', 'imdb_rating', 'imdb_numVotes',
       'tmdb_rating', 'trakt_rating', 'trakt_numVotes', 'metacritic_rating',
       'metacritic_numVotes'],
      dtype='object')
        tconst    title  startYear  imdb_rating  imdb_numVotes  tmdb_rating  \
777  tt9288030  Reacher       2022          8.1         154886         80.0   

     t